<a href="https://colab.research.google.com/github/dilliprasadtathireddi/Recommendation-System/blob/main/recommandation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Recommandation System**

In [ ]:
#Importing Libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
users = pd.read_csv('/content/Users.csv')
books = pd.read_csv('/content/Books.csv')
ratings = pd.read_csv('/content/Ratings.csv')


In [ ]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
users.shape

(278858, 3)

In [ ]:
books.shape

(113408, 8)

In [ ]:
ratings.shape

(1149780, 3)

**Data Pre-Processing**




Let's check if there is any null values in three datasets

In [ ]:
#users.isnull().count()
users.isnull().sum()

,0
User-ID,0
Location,0
Age,110762


In [ ]:
users = users.dropna()

In [ ]:
users.isnull().sum()

,0
User-ID,0
Location,0
Age,0


In [ ]:
users.shape

(168096, 3)

In [ ]:
books.isnull().sum()

,0
ISBN,0
Book-Title,0
Book-Author,0
Year-Of-Publication,0
Publisher,0
Image-URL-S,0
Image-URL-M,1
Image-URL-L,1


In [ ]:
#books.dropna(inplace=True)
#or
books = books.dropna()

In [ ]:
books.isnull().sum()

,0
ISBN,0
Book-Title,0
Book-Author,0
Year-Of-Publication,0
Publisher,0
Image-URL-S,0
Image-URL-M,0
Image-URL-L,0


In [ ]:
books.shape

(113407, 8)

In [ ]:
ratings.isnull().sum()

,0
User-ID,0
ISBN,0
Book-Rating,0


**Now Let's lookup for Duplicates values**

In [ ]:
users.duplicated().sum()

0

In [ ]:
books.duplicated().sum()

0

In [ ]:
ratings.duplicated().sum()

0

Unique Count of each set

In [ ]:
users.nunique()

,0
User-ID,168096
Location,41253
Age,165


In [ ]:
books.nunique()

,0
ISBN,113407
Book-Title,102929
Book-Author,47035
Year-Of-Publication,94
Publisher,8716
Image-URL-S,113348
Image-URL-M,113348
Image-URL-L,113348


In [ ]:
ratings.nunique()

,0
User-ID,105283
ISBN,340556
Book-Rating,11


In [ ]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
np.sort(ratings['Book-Rating'].unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [ ]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113407 entries, 0 to 113406
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 113407 non-null  object
 1   Book-Title           113407 non-null  object
 2   Book-Author          113407 non-null  object
 3   Year-Of-Publication  113407 non-null  int64 
 4   Publisher            113407 non-null  object
 5   Image-URL-S          113407 non-null  object
 6   Image-URL-M          113407 non-null  object
 7   Image-URL-L          113407 non-null  object
dtypes: int64(1), object(7)
memory usage: 7.8+ MB


In [ ]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [ ]:
books['Year-Of-Publication'] = books['Year-Of-Publication'].astype('int32')

In [ ]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113407 entries, 0 to 113406
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 113407 non-null  object
 1   Book-Title           113407 non-null  object
 2   Book-Author          113407 non-null  object
 3   Year-Of-Publication  113407 non-null  int32 
 4   Publisher            113407 non-null  object
 5   Image-URL-S          113407 non-null  object
 6   Image-URL-M          113407 non-null  object
 7   Image-URL-L          113407 non-null  object
dtypes: int32(1), object(7)
memory usage: 7.4+ MB


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168096 entries, 1 to 278854
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   168096 non-null  int64  
 1   Location  168096 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 5.1+ MB


# **Popularity Based Recommandation System**

In [ ]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
users.head()

,User-ID,Location,Age
1,2,"stockton, california, usa",18.0
3,4,"porto, v.n.gaia, portugal",17.0
5,6,"santa monica, california, usa",61.0
9,10,"albacete, wisconsin, spain",26.0
10,11,"melbourne, victoria, australia",14.0


In [ ]:
#Merging Data Frames(User and Books)
books_with_ratings = ratings.merge(books, on='ISBN')

In [ ]:
books_with_ratings.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [ ]:
books_with_ratings.shape

(789778, 10)

In [ ]:
popular_books = books_with_ratings.groupby('Book-Title').agg(num_rating = ('Book-Rating', 'count'), avg_rating = ('Book-Rating', 'mean'))

In [ ]:
popular_books.reset_index()

,Book-Title,num_rating,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
2,Earth Prayers From around the World: 365 Pray...,10,5.000000
3,Final Fantasy Anthology: Official Strategy Gu...,4,5.000000
4,Good Wives: Image and Reality in the Lives of...,10,3.200000
...,...,...,...
102873,Ã?Â?berallnie. AusgewÃ?Â¤hlte Gedichte 1928 - ...,1,10.000000
102874,Ã?Â?bermorgen.,1,0.000000
102875,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
102876,Ã?Â?stlich der Berge.,3,2.666667


In [ ]:
popular_books.head()

,num_rating,avg_rating
Book-Title,,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",4,2.25
Beyond IBM: Leadership Marketing and Finance for the 1990s,1,0.00
"Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth",10,5.00
Final Fantasy Anthology: Official Strategy Guide (Brady Games),4,5.00
"Good Wives: Image and Reality in the Lives of Women in Northern New England, 1650-1750",10,3.20


In [ ]:
popular_books.sort_values('num_rating', ascending=False)

,num_rating,avg_rating
Book-Title,,
Wild Animus,2502,1.019584
The Lovely Bones: A Novel,1295,4.468726
The Da Vinci Code,894,4.652125
A Painted House,831,3.220217
The Nanny Diaries: A Novel,828,3.530193
...,...,...
How to eat better and spend less: A complete guide to vegetable gardening,1,0.000000
The Boy With the Helium Head,1,6.000000
The Boy and the Tigers,1,0.000000


In [ ]:
popular_books = popular_books[popular_books['num_rating']>300].sort_values('avg_rating', ascending = False)

In [ ]:
popular_books

,num_rating,avg_rating
Book-Title,,
Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
Harry Potter and the Goblet of Fire (Book 4),387,5.824289
Harry Potter and the Order of the Phoenix (Book 5),346,5.488439
Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
"The Fellowship of the Ring (The Lord of the Rings, Part 1)",354,4.906780
...,...,...
While I Was Gone,431,2.580046
The Perfect Storm : A True Story of Men Against the Sea,403,2.526055
A Map of the World,325,2.483077


In [ ]:
popular_books.describe()

,num_rating,avg_rating
count,118.000000,118.000000
mean,473.500000,3.496165
std,242.907275,0.708114
min,301.000000,1.019584
25%,343.000000,3.032785
50%,427.500000,3.399018
75%,523.500000,3.776174
max,2502.000000,5.852804


In [ ]:
popular_books = popular_books.head(50)

In [ ]:
popular_books

,num_rating,avg_rating
Book-Title,,
Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
Harry Potter and the Goblet of Fire (Book 4),387,5.824289
Harry Potter and the Order of the Phoenix (Book 5),346,5.488439
Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
"The Fellowship of the Ring (The Lord of the Rings, Part 1)",354,4.906780
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),575,4.895652
To Kill a Mockingbird,503,4.695825
The Da Vinci Code,894,4.652125
The Five People You Meet in Heaven,427,4.543326


In [ ]:
popular_books = popular_books.merge(books , on='Book-Title').drop_duplicates('Book-Title')[['Book-Title',
                                                                                           'Book-Author',
                                                                                           'Image-URL-M',
                                                                                           'num_rating',
                                                                                           'avg_rating'

                                                                                           ]]

In [ ]:
popular_books

,Book-Title,Book-Author,Image-URL-M,num_rating,avg_rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
5,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,346,5.488439
8,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
11,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,354,4.906780
16,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.895652
18,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,503,4.695825
23,The Da Vinci Code,Dan Brown,http://images.amazon.com/images/P/0385504209.0...,894,4.652125
26,The Five People You Meet in Heaven,Mitch Albom,http://images.amazon.com/images/P/0786868716.0...,427,4.543326
27,The Catcher in the Rye,J.D. Salinger,http://images.amazon.com/images/P/0316769487.0...,442,4.536199


# **Collaborative Filtering**

Similar book prediction based on user feedback

In [ ]:
books_with_ratings.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [ ]:
#grouping based on userid
x = books_with_ratings.groupby('User-ID').count()

In [ ]:
x

,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
User-ID,,,,,,,,,
2,1,1,1,1,1,1,1,1,1
8,17,17,17,17,17,17,17,17,17
9,3,3,3,3,3,3,3,3,3
10,1,1,1,1,1,1,1,1,1
12,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
278846,1,1,1,1,1,1,1,1,1
278849,2,2,2,2,2,2,2,2,2
278851,12,12,12,12,12,12,12,12,12


In [ ]:
x.index

Index([     2,      8,      9,     10,     12,     14,     16,     17,     19,
           20,
       ...
       278832, 278836, 278838, 278843, 278844, 278846, 278849, 278851, 278852,
       278854],
      dtype='int64', name='User-ID', length=81308)

In [ ]:
x.shape

(81308, 9)

In [ ]:
x = x['Book-Rating']>200

In [ ]:
x

,Book-Rating
User-ID,
2,False
8,False
9,False
10,False
12,False
...,...
278846,False
278849,False
278851,False


In [ ]:
x.shape

(81308,)

In [ ]:
x.index

Index([     2,      8,      9,     10,     12,     14,     16,     17,     19,
           20,
       ...
       278832, 278836, 278838, 278843, 278844, 278846, 278849, 278851, 278852,
       278854],
      dtype='int64', name='User-ID', length=81308)

In [ ]:
power_users = x[x].index

In [ ]:
power_users

Index([   254,   2276,   2766,   2977,   3363,   4017,   4385,   6251,   6323,
         6543,
       ...
       271448, 271705, 273979, 274004, 274061, 274301, 274308, 275970, 277427,
       278418],
      dtype='int64', name='User-ID', length=585)

In [ ]:
power_users.sort_values()

Index([   254,   2276,   2766,   2977,   3363,   4017,   4385,   6251,   6323,
         6543,
       ...
       271448, 271705, 273979, 274004, 274061, 274301, 274308, 275970, 277427,
       278418],
      dtype='int64', name='User-ID', length=585)

In [ ]:
filtered = books_with_ratings[books_with_ratings['User-ID'].isin(power_users)]

In [ ]:
filtered

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
5,23768,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
7,28523,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
15,77940,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
16,81977,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
789536,275970,1880837927,0,The Theology of the Hammer,Millard Fuller,1994,Smyth &amp; Helwys Publishing,http://images.amazon.com/images/P/1880837927.0...,http://images.amazon.com/images/P/1880837927.0...,http://images.amazon.com/images/P/1880837927.0...
789537,275970,188717897X,0,The Ordeal of Integration: Progress and Resent...,Orlando Patterson,1998,Civitas Book Publisher,http://images.amazon.com/images/P/188717897X.0...,http://images.amazon.com/images/P/188717897X.0...,http://images.amazon.com/images/P/188717897X.0...
789538,275970,1888889047,0,Pushcart's Complete Rotten Reviews &amp; Rejec...,Bill Henderson,1998,Pushcart Press,http://images.amazon.com/images/P/1888889047.0...,http://images.amazon.com/images/P/1888889047.0...,http://images.amazon.com/images/P/1888889047.0...
789539,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),http://images.amazon.com/images/P/1931868123.0...,http://images.amazon.com/images/P/1931868123.0...,http://images.amazon.com/images/P/1931868123.0...


In [ ]:
y = filtered.groupby('Book-Title').count()

In [ ]:
y

,User-ID,ISBN,Book-Rating,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
Book-Title,,,,,,,,,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",1,1,1,1,1,1,1,1,1
Beyond IBM: Leadership Marketing and Finance for the 1990s,1,1,1,1,1,1,1,1,1
"Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth",1,1,1,1,1,1,1,1,1
Final Fantasy Anthology: Official Strategy Guide (Brady Games),2,2,2,2,2,2,2,2,2
"Good Wives: Image and Reality in the Lives of Women in Northern New England, 1650-1750",4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...
"Â¿QuÃ© me quieres, amor?",1,1,1,1,1,1,1,1,1
"Ã?ber den Wunsch, sich wohlzufÃ¼hlen: Geschichten",1,1,1,1,1,1,1,1,1
Ã?ngeles fugaces (Falling Angels),1,1,1,1,1,1,1,1,1


In [ ]:
y.sort_values('User-ID', ascending= False)

,User-ID,ISBN,Book-Rating,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
Book-Title,,,,,,,,,
Wild Animus,254,254,254,254,254,254,254,254,254
Bridget Jones's Diary,222,222,222,222,222,222,222,222,222
The Lovely Bones: A Novel,216,216,216,216,216,216,216,216,216
The Notebook,209,209,209,209,209,209,209,209,209
Divine Secrets of the Ya-Ya Sisterhood: A Novel,191,191,191,191,191,191,191,191,191
...,...,...,...,...,...,...,...,...,...
Making Peace With Your Past: The Six Essential Steps To Enjoying A Great Future,1,1,1,1,1,1,1,1,1
Making Scenes,1,1,1,1,1,1,1,1,1
Making Schools Smarter : A System for Monitoring School and District Progress,1,1,1,1,1,1,1,1,1


In [ ]:
y = y['User-ID']>50

In [ ]:
y

,User-ID
Book-Title,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",False
Beyond IBM: Leadership Marketing and Finance for the 1990s,False
"Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth",False
Final Fantasy Anthology: Official Strategy Guide (Brady Games),False
"Good Wives: Image and Reality in the Lives of Women in Northern New England, 1650-1750",False
...,...
"Â¿QuÃ© me quieres, amor?",False
"Ã?ber den Wunsch, sich wohlzufÃ¼hlen: Geschichten",False
Ã?ngeles fugaces (Falling Angels),False


In [ ]:
famous_books = y[y].index

In [ ]:
famous_books

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Fine Balance',
       'A Heartbreaking Work of Staggering Genius',
       ...
       'Wild Animus', 'Winter Moon', 'Winter Solstice', 'Wish You Well',
       'Without Remorse', 'Wuthering Heights', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=543)

In [ ]:
final_ratings = filtered[filtered['Book-Title'].isin(famous_books)]

In [ ]:
final_ratings

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
61,278418,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
63,3363,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
64,7158,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
67,11676,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
72,23768,0446520802,6,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
...,...,...,...,...,...,...,...,...,...,...
784279,264317,0833563505,0,The Queen of the Damned (Vampire Chronicles (P...,Anne Rice,1999,Sagebrush Bound,http://images.amazon.com/images/P/0833563505.0...,http://images.amazon.com/images/P/0833563505.0...,http://images.amazon.com/images/P/0833563505.0...
785563,266865,0531001725,10,The Catcher in the Rye,Jerome David Salinger,1973,Scholastic Library Pub,http://images.amazon.com/images/P/0531001725.0...,http://images.amazon.com/images/P/0531001725.0...,http://images.amazon.com/images/P/0531001725.0...
787502,271284,0440910927,0,The Rainmaker,John Grisham,1995,Island,http://images.amazon.com/images/P/0440910927.0...,http://images.amazon.com/images/P/0440910927.0...,http://images.amazon.com/images/P/0440910927.0...
787794,271705,B0001PIOX4,0,Fahrenheit 451,Ray Bradbury,1993,Simon &amp; Schuster,http://images.amazon.com/images/P/B0001PIOX4.0...,http://images.amazon.com/images/P/B0001PIOX4.0...,http://images.amazon.com/images/P/B0001PIOX4.0...


In [ ]:
pivot_table = final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')

In [ ]:
pivot_table

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271448,271705,273979,274004,274061,274301,274308,275970,277427,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Bend in the Road,0.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [ ]:
pivot_table.fillna(0, inplace = True)

In [ ]:
pivot_table

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271448,271705,273979,274004,274061,274301,274308,275970,277427,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
similarity_score = cosine_similarity(pivot_table)

In [ ]:
similarity_score_df = pd.DataFrame(similarity_score)

In [ ]:
similarity_score_df

,0,1,2,3,4,5,6,7,8,9,...,533,534,535,536,537,538,539,540,541,542
0,1.000000,0.130385,0.015559,0.000000,0.066993,0.034452,0.108869,0.179625,0.042755,0.082491,...,0.022177,0.043536,0.062011,0.016065,0.012652,0.011425,0.020630,0.147617,0.093942,0.053030
1,0.130385,1.000000,0.238607,0.000000,0.112795,0.117237,0.146107,0.229594,0.051436,0.089316,...,0.097763,0.090908,0.073421,0.101882,0.168060,0.074748,0.210600,0.084915,0.200643,0.163958
2,0.015559,0.238607,1.000000,0.000000,0.080870,0.122939,0.000000,0.132156,0.045566,0.000000,...,0.136749,0.130869,0.170408,0.241921,0.024380,0.022014,0.181092,0.052109,0.059212,0.124392
3,0.000000,0.000000,0.000000,1.000000,0.000000,0.122706,0.000000,0.000000,0.000000,0.157394,...,0.025389,0.000000,0.000000,0.000000,0.000000,0.137328,0.078723,0.000000,0.000000,0.000000
4,0.066993,0.112795,0.080870,0.000000,1.000000,0.115553,0.117160,0.047974,0.000000,0.006905,...,0.096658,0.091360,0.083073,0.106326,0.026551,0.029507,0.105453,0.044802,0.132511,0.017158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,0.011425,0.074748,0.022014,0.137328,0.029507,0.178741,0.000000,0.209795,0.000000,0.031125,...,0.031379,0.061600,0.022614,0.022731,0.017902,1.000000,0.029190,0.030208,0.043479,0.053218
539,0.020630,0.210600,0.181092,0.078723,0.105453,0.127148,0.092038,0.058411,0.169927,0.000000,...,0.116474,0.296623,0.205534,0.193072,0.032327,0.029190,1.000000,0.054548,0.213102,0.200948
540,0.147617,0.084915,0.052109,0.000000,0.044802,0.105557,0.050397,0.060449,0.000000,0.000000,...,0.114488,0.115115,0.042260,0.061359,0.033455,0.030208,0.054548,1.000000,0.081251,0.021620
541,0.093942,0.200643,0.059212,0.000000,0.132511,0.131117,0.191496,0.087004,0.096424,0.000000,...,0.084401,0.165686,0.123387,0.061141,0.048152,0.043479,0.213102,0.081251,1.000000,0.122564


In [ ]:
pivot_table.index

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Fine Balance',
       'A Heartbreaking Work of Staggering Genius',
       ...
       'Wild Animus', 'Winter Moon', 'Winter Solstice', 'Wish You Well',
       'Without Remorse', 'Wuthering Heights', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=543)

In [ ]:
similarity_score_df.index = pivot_table.index

In [ ]:
similarity_score_df.columns = pivot_table.index

In [ ]:
similarity_score_df

Book-Title,1984,1st to Die: A Novel,2nd Chance,4 Blondes,A Bend in the Road,A Case of Need,"A Child Called \It\"": One Child's Courage to Survive""",A Civil Action,A Fine Balance,A Heartbreaking Work of Staggering Genius,...,Wild Animus,Winter Moon,Winter Solstice,Wish You Well,Without Remorse,Wuthering Heights,You Belong To Me,Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zoya,"\O\"" Is for Outlaw"""
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,1.000000,0.130385,0.015559,0.000000,0.066993,0.034452,0.108869,0.179625,0.042755,0.082491,...,0.022177,0.043536,0.062011,0.016065,0.012652,0.011425,0.020630,0.147617,0.093942,0.053030
1st to Die: A Novel,0.130385,1.000000,0.238607,0.000000,0.112795,0.117237,0.146107,0.229594,0.051436,0.089316,...,0.097763,0.090908,0.073421,0.101882,0.168060,0.074748,0.210600,0.084915,0.200643,0.163958
2nd Chance,0.015559,0.238607,1.000000,0.000000,0.080870,0.122939,0.000000,0.132156,0.045566,0.000000,...,0.136749,0.130869,0.170408,0.241921,0.024380,0.022014,0.181092,0.052109,0.059212,0.124392
4 Blondes,0.000000,0.000000,0.000000,1.000000,0.000000,0.122706,0.000000,0.000000,0.000000,0.157394,...,0.025389,0.000000,0.000000,0.000000,0.000000,0.137328,0.078723,0.000000,0.000000,0.000000
A Bend in the Road,0.066993,0.112795,0.080870,0.000000,1.000000,0.115553,0.117160,0.047974,0.000000,0.006905,...,0.096658,0.091360,0.083073,0.106326,0.026551,0.029507,0.105453,0.044802,0.132511,0.017158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights,0.011425,0.074748,0.022014,0.137328,0.029507,0.178741,0.000000,0.209795,0.000000,0.031125,...,0.031379,0.061600,0.022614,0.022731,0.017902,1.000000,0.029190,0.030208,0.043479,0.053218
You Belong To Me,0.020630,0.210600,0.181092,0.078723,0.105453,0.127148,0.092038,0.058411,0.169927,0.000000,...,0.116474,0.296623,0.205534,0.193072,0.032327,0.029190,1.000000,0.054548,0.213102,0.200948
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.147617,0.084915,0.052109,0.000000,0.044802,0.105557,0.050397,0.060449,0.000000,0.000000,...,0.114488,0.115115,0.042260,0.061359,0.033455,0.030208,0.054548,1.000000,0.081251,0.021620


In [ ]:
def recommandation(book_name):
  index = np.where(pivot_table.index==book_name)[0][0]
  similar_items = sorted(list(enumerate(similarity_score[index])), key=lambda x:x[1], reverse=True)[1:6]
  data_list = []
  for i in similar_items:
    item = []
    temp_df = books[books['Book-Title']==pivot_table.index[i[0]]]
    item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
    item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
    item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
    data_list.append(item)
  return data_list

In [ ]:
recommandation('Animal Farm')

[['1984',
  'George Orwell',
  'http://images.amazon.com/images/P/0451524934.01.MZZZZZZZ.jpg'],
 ['Call of the Wild',
  'Jack London',
  'http://images.amazon.com/images/P/1559029838.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg'],
 ['American Gods',
  'Neil Gaiman',
  'http://images.amazon.com/images/P/0380789035.01.MZZZZZZZ.jpg'],
 ['Pride and Prejudice',
  'Jane Austen',
  'http://images.amazon.com/images/P/055321215X.01.MZZZZZZZ.jpg']]

In [ ]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

# **Exporting Data and model to Pickle(pkl)**

In [91]:
import pickle as pkl
pkl.dump(popular_books, open('popular_books.pkl', 'wb'))

In [92]:
pkl.dump(books, open('books.pkl', 'wb'))

In [93]:
pkl.dump(pivot_table, open('pivot_table.pkl', 'wb'))

In [94]:
pkl.dump(similarity_score, open('similarity_scores.pkl', 'wb'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')